In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_parquet('seoul_final.parquet')

In [3]:
df

,Duration,Distance,Haversine,Phour,Dhour,Temp,GroundTemp
0,3,50,0.000000,0,0,-3.2,-2.2
1,24,7670,5.713529,0,0,-3.2,-2.2
2,8,1390,0.913702,0,0,-3.2,-2.2
3,8,1820,1.468027,0,0,-3.2,-2.2
4,4,850,0.823227,0,0,-3.2,-2.2
...,...,...,...,...,...,...,...
9601134,67,6930,0.000000,23,0,-5.2,-5.1
9601135,58,8320,4.402263,23,0,-5.2,-5.1
9601136,118,2730,1.389156,22,0,-5.4,-5.0
9601137,90,17170,8.779306,23,1,-5.2,-5.1


In [4]:
from sklearn.model_selection import train_test_split

In [5]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

In [6]:
X = df.drop('Duration', axis =1).values
y = df.Duration.values

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [8]:
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

In [9]:
X_train_scaled

array([[-0.66548145, -0.54153806,  0.57948285,  0.55103666,  1.3725609 ,
         1.1041207 ],
       [-0.73117036, -0.62988424, -0.06077585, -0.06598113,  1.0145342 ,
         1.9813895 ],
       [-0.11723135,  0.22036551, -1.021164  , -0.99150777, -1.0809753 ,
        -0.92703366],
       ...,
       [ 0.75441045,  1.0957931 , -1.8214874 , -1.6085255 ,  0.15105782,
        -0.09718465],
       [ 0.8352584 ,  2.0149584 , -0.5409699 , -0.52874446, -0.14378764,
         0.456048  ],
       [-0.03133042, -0.60608137, -0.06077585,  0.08827332,  0.07734636,
         0.40862805]], dtype=float32)

In [10]:
X_train_scaled.shape

(6720797, 6)

In [11]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [12]:
lr.fit(X_train_scaled, y_train)

LinearRegression()

In [13]:
pred = lr.predict(X_test_scaled)

In [14]:
from sklearn.metrics import r2_score

In [15]:
r2_score(y_test, pred)

0.5642614753073121

In [16]:
from sklearn.metrics import mean_squared_error

In [17]:
mean_squared_error(y_test, pred)

273.16583

In [18]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

In [19]:
rf.fit(X_train_scaled, y_train)

KeyboardInterrupt: 

In [ ]:
pred = rf.predict(X_test_scaled)

In [ ]:
r2_score(y_test, pred)

In [21]:
from lightgbm import LGBMRegressor

In [22]:
lgbm = LGBMRegressor()

In [23]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

In [26]:
from numpy import mean
from numpy import std

In [24]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(lgbm, X_train_scaled, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')

In [27]:
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

MAE: -6.527 (0.014)


In [36]:
lgbm.fit(X_train_scaled, y_train)

LGBMRegressor()

In [37]:
lgbm.score(X_train_scaled, y_train)

In [32]:
import pickle

In [33]:
filename = 'finalized_model.sav'
pickle.dump(lgbm, open(filename, 'wb'))

In [35]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test_scaled, y_test)
print(result)

NotFittedError: Estimator not fitted, call `fit` before exploiting the model.